# Comparing differences

- Let's say we want to compare two commits
    - Commit 1: d44e90
    - Commit 2: 2dc131b
        - This is our most recent commit
    
- To see an outline of their differences, we use the following command:

```shell
git diff d44e90 HEAD
```

- As we can see, since 2dc131b is our most recent commit ID, we use HEAD instead

_____

# Branching and merge types

- **Branches** are simply the names we assign to a timeline of commits
    - Effectively a labelling system
    
- So far, we've only been working on the master branch
    - As we'll see, we can create a **feature branch** to make some updates without messing with our main production code
        - When we're satisfied with our new feature, we can **merge** the feature branch back into the master branch
        
### Types of Merges

1. **Fast-forward merge**
    - Simplest case
        - Occurs when nothing new has been added to the master branch while we've been working on the feature branch
    - As if we never created a new branch (and we were working on the master branch the whole time)
        - The commits are moved from the feature branch to the master branch
    - Can be disabled
    
2. **Automatic merge**
    - Occurs when changes have been made to the master branch while we've been working on the feature branch
        - The changes, however, don't cause problems with one another
    - Both timelines are preserved
    
3. **Manual merge**
    - This is the trickiest case
        - Occurs when changes made in one branch interfere with changes made in another
            - That means that automatic merging is impossible
    - The owner needs to decide which code to keep, and which to discard

___

# Special markers

- **HEAD**
    - Refers to the most recent commit to the branch we're on
        - With a more advanced command, we can move the HEAD of the branch to another commit

____

# Simple branching example

- Let's say we're on the master branch, and we've made some changes to the README.md file
    - Before we commit this update to master, we want to create a new branch and commit it to there first
        - To **create a new branch and switch over to it**, we use the following command:
        
```shell
git checkout -b update_branch
```

- The message returned will read:

```
M    README.md
Switched to a new branch 'update_branch'
```

- This means that not only are we on the new branch, but we've transfered the changes we made in the working directory to the new branch
    - This is useful if we're working on our master branch, and we decide that it's probably a good idea to commit these changes to a separate branch first (just to be safe)
    
- Now, we use `git add .` and `git commit -m 'adding updates to branch'` to move our changes to the staging area of our new branch
    - If we want to see the differences between branches (instead of commits, like we saw recently), we can specify the branch names:
    
```shell
git diff update_branch master
```

- Next, let's say we're happy with our updates, and want to merge the updates branch onto the master branch
    - First, we need to switch back over to our master branch using the following command:
    
```shell
git checkout master
```

- The message returned will read:

```shell
Switched to branch 'master'
```

- Now, to merge the branches, we use the following command:

```shell
git merge update_branch
```

- Since this merge is so simple, the commit message will tell us it's a *fast-forward merge*

- We don't need update_branch (since it's basically just a series of tags), so we can delete it wil the following command:

```shell
git branch -d update_branch
```

- **Note:** this doesn't mean that the history gets deleted, just that they're now associated with the master (and we removed duplicated tags)

___

# Conflict resolution

- Let's say we have two branches
    1. master
        - This is our main branch with all our production code
    2. very-bad
        - This is a feature branch we made, whose files conflict with those on the master branch
            - **We'll need to manually solve the conflicts when we try to merge the two branches**
            
- While on the master branch, we try to merge in our very-bad branch with the following command:

```shell
git merge very-bad
```

- This returns the message

```
Automatic merge failed; fix conflicts and then commit the result
```

- We're now in a *merging state*
    - The name of our current branch in our terminal will switch from `master` to `master|MERGING`
    
- We'll use the merge tool to solve the conflicts

```shell
git mergetool
```

- Once we've resolved the conflict, we commit it

```shell
git commit -m "Resolving conflict"
```

- **Note**: our repo will now contain our original file (before we made the changes) as a .orig file
    - If we don't want to include it in our repo, we can add `*.orig` to our .gitignore file

___

# Marking special events with tagging

- Let's say that we've reached a certain milestone in our code
    - We want to mark the milestone by **adding a tag to a commit id**
    
- There are two types of tags:
    1. Lightweight tags
        - We simply need to specify the tag name
        - There is no information associated with the tag
    2. Annotated tags
        - Contains additional information
        
- To add a **lightweight** tag, we use the following command:

```shell
git tag my_tag
```

- This will assign the tag to the current HEAD
- To view the tags, we use the command:

```shell
git tag --list
```

- To add an **annotated** tag, we use the following command:

```shell
git tag -a my_tag -m "Commit message"
```

- Now, to view the annotation for the tag, we use the command:

```shell
git show my_tag
```

_____

# Stashing work-in-progress

- Let's say we have some uncommitted changes in our working directory
    - **We want to keep this work, but don't want to commit it right now, or create a new branch**
        - We can use *git stashing*
        
- We stash our current files using the following command

```shell
git stash
```

- Now, we can see all our stashes using the command:

```shell
git stash list
```

- After we've stashed our changes, if we run `git status`, we'll get the message that there's nothing to commit
    - i.e. we're back on a clean working directory
    
- Once we've made our updates, we want to **reload our changes from the stash**:
    - The following command reloads our most recent stash, and drops it from our stored stashes
    
```shell
git stash pop
```

- If we run `git stash list`, the stash will no longer be listed
    - Also, if we run `git status`, we'll have our file changes again

____

# Time travel with reset and reflog

## Reset

**Types of reset**

1. soft
    - least destructive
    - essentially changes where HEAD is pointing
2. mixed
    - default
3. hard
    - most destrutive
    
### Example

- Most recent commit ID: CD456
    - This is currently our branch's HEAD
- Changes since CD456
    - In staging area:
        - Made a change to FILE_2
    - In working directory:
        - Made a change to FILE_3       
- Commit ID before CD456: AB123
    - Changes between AB123 and CD456:
        - Made a change to FILE_1, committed to repo
    
![](images/commit_history.png)
    
**Example 1 - soft reset**
    
```shell
git reset AB123 --soft
```

- After this reset:
    - In staging area:
        - Changes to FILE_1
        - Changes to FILE_2
    - In working directory:
        - Changes to FILE_3
        
- As we can see, everything that was added (by `git add FILE_1` and `git add FILE_2`) is now in the staging area
    - Everything that has changed, but hasn't been added to the staging area is in the working directory
    - **So what is this soft reset really doing?**
        - Undoing commit CD456, but preserving all changes since then
        
**Example 2 - mixed reset**

- This reset is similar to soft, except it **unstages the changes we made between the commits**

- After this reset:
    - In staging area:
        - Changes to FILE_1
    - In working directory:
        - Changes to FILE_2
        - Changes to FILE_3
        
- As we can see, the changes to FILE_2 are now unstaged
    - Whereas in the soft commit, they remained in the staging area
    
**Example 3 - hard reset**

- This **unstages all changes since the commit AB123**
    - We essentially revert back to how the repo was right after the commit

- After this reset:
    - In staging area:
    - In working directory:
        - Changes to FILE_1
        - Changes to FILE_2
        - Changes to FILE_3
        
- As we can see, the all changes are now unstaged

## Reflog

**Git log vs. git reflog**

- `git log` tells us all our commit IDs
    - `git reflog` tells us all the different actions we've taken in our repository
    
- We can use this to get to the specific git ID we want